# ASEWA - Visualization
Visualizing layers, exploring weights, perform mass transfer
---
Janina Klarmann, Laura Kühl

In [ ]:
import tensorflow as tf
import numpy as np
import os
from torchvision import transforms
import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
import math

import PIL.Image
import IPython.display as display
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12, 12)
mpl.rcParams['axes.grid'] = False

from keras.models import Model
import glob

In [ ]:
vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
vgg.trainable = False

In [ ]:
# Define a function to load an image and limit its maximum dimension to 512 pixels.
def load_img(path_to_img):
    '''Load and preprocess an image from a path
    Input: Image path
    Ouput: preprocessed image to a tensor'''
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    
    return img



def imshow(image, title=None):
    '''For image visualisation'''
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)
    


In [ ]:
from tensorflow.python.tools import module_util
def VGG19_Model(layer_names):
    """ Creates our model with access to intermediate layers. 

    This function will load the VGG19 model and access the intermediate layers. 
    These layers will then be used to create a new model that will take input image
    and return the outputs from these intermediate layers from the VGG model. 

    Returns:
    returns a keras model that takes image inputs and outputs the style and 
      content intermediate layers. 
    """
    # Load our model. We load pretrained VGG, trained on imagenet data (weights=’imagenet’)
    vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False    

    # Get output layers corresponding to style and content layers 
    outputs = [vgg.get_layer(name).output for name in layer_names]
    # Build model 
    model = tf.keras.Model([vgg.input], outputs)


    return model 

In [ ]:
class StyleExtractionModel (tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleExtractionModel, self).__init__()
        self.vgg19 = VGG19_Model(style_layers+content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg19.trainable = False 

    def gram_matrix(self, input_tensor):
        result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
        input_shape = tf.shape(input_tensor)
        num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
        return result/(num_locations)

    def call(self, inputs):
        "Expects float input in [0,1]"
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg19(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [self.gram_matrix(style_output)
                        for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}






class StyleTrainingModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleTrainingModel, self).__init__()
        self.ExtractionModel = StyleExtractionModel(style_layers, content_layers)
        self.opt = tf.optimizers.Adam(learning_rate=10, beta_1=0.99, epsilon=1e-1)
        self.style_weight = 1000
        self.content_weight = 1e6
        # use when doing the weight exploration
        self.style_targets = self.ExtractionModel(style_image)['style']
        self.content_targets = self.ExtractionModel(content_image)['content']
        self.num_style_layers = len(style_layers)
        self.num_content_layers = len(content_layers)

    def clip_0_1(self, image):
        return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

    def tensor_to_image(self, tensor):
        '''Takes a Tensor as an inout and plots you an image'''
        tensor = tensor*255
        tensor = np.array(tensor, dtype=np.uint8)
        if np.ndim(tensor)>3:
            assert tensor.shape[0] == 1
            tensor = tensor[0]
        return PIL.Image.fromarray(tensor)
    
    def style_content_loss(self, outputs, style_targets):
        '''Calculate the loss from the style and content layers
        returns the overall loss'''
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                               for name in style_outputs.keys()])
                
        style_loss *= self.style_weight / self.num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-self.content_targets[name])**2) 
                                 for name in content_outputs.keys()])
        content_loss *= self.content_weight / self.num_content_layers

        loss = style_loss + content_loss
        
        return loss

    
    #@tf.function()
    def train_step(self, img, art_image):
        '''Inputs a style image and an input image. To perform style transfer on the input.'''
        style_targets = self.ExtractionModel(art_image)['style']
        with tf.GradientTape() as tape:
            outputs = self.ExtractionModel(img)
            loss = self.style_content_loss(outputs, style_targets)
        grad = tape.gradient(loss, img)
        self.opt.apply_gradients([(grad, img)])
        image.assign(self.clip_0_1(img))

        
### Weight_Exploration###########################################################################################
# Use this call and style_content_loss function when doing the weight exploration

    def style_content_loss_weight_exploration(self, outputs, style_weight, content_weight):
        '''Takes a style and content weight. Calculate the loss from the style and content layers
        returns the overall loss'''
        style_outputs = outputs['style']
        content_outputs = outputs['content']
        style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-self.style_targets[name])**2) 
                               for name in style_outputs.keys()])
                
        style_loss *= style_weight / self.num_style_layers

        content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-self.content_targets[name])**2) 
                                 for name in content_outputs.keys()])
        content_loss *= content_weight / self.num_content_layers

        loss = style_loss + content_loss
        
        return loss

    #@tf.function()
    def weight_exploration(self, img, style_weight, content_weight):
        '''Takes a style and content weight in addition to the input image'''
        with tf.GradientTape() as tape:
            outputs = self.ExtractionModel(img)
            loss = self.style_content_loss_weight_exploration(outputs, style_weight, content_weight)
        grad = tape.gradient(loss, img)
        self.opt.apply_gradients([(grad, img)])
        image.assign(self.clip_0_1(img))

In [ ]:
#Van Gogh
style_path = tf.keras.utils.get_file('Vincent-van-Gogh-Die-Sternennacht-Detail.jpg','https://artinwords.de/wp-content/uploads/Vincent-van-Gogh-Die-Sternennacht-Detail.jpg')

#from training data
#style_path = '../input/art-movements/dataset/train/expressionism/expressionismo_202.jpg'

#house
content_path = '../input/chchhouse/photo_2022-04-05_00-54-40.jpg'


#kitten
#content_path = tf.keras.utils.get_file('photo-1592194996308-7b43878e84a6?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1887&q=80','https://images.unsplash.com/photo-1592194996308-7b43878e84a6?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1887&q=80')

#Singapur park
# content_path = tf.keras.utils.get_file('garten-singapur-m-06435870-jpg--82522-.jpg', 'https://image.geo.de/30144962/t/Jr/v4/w1440/r1.7778/-/garten-singapur-m-06435870-jpg--82522-.jpg')

In [ ]:
#load a content and style image
content_image = load_img(content_path)
style_image = load_img(style_path)

# create a white noise image, it has to have the size of the content image
im = tf.random.uniform((288, 512))
white_noise_image = tf.stack([im, im, im], -1)
white_noise_image = tf.expand_dims(white_noise_image, 0)
white_noise_image.shape


#plot all three images
plt.subplot(2, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(2, 2, 2)
imshow(style_image, 'Style Image')

plt.subplot(2, 2, 3)
imshow(white_noise_image)

In [ ]:
# define the style and content layer
content_layer = ['block4_conv1']
style_layer_names = ['block1_conv1', 'block2_conv1','block3_conv1','block4_conv1', 'block5_conv1']

# instantiate the model with the choosen layer
MyModel = StyleTrainingModel(style_layer_names, content_layer)

#the inputimage that gets augmented
image = tf.Variable(content_image)
# image = tf.Variable(random_noise)

In [ ]:
# perform a style transfer, with the input and the style image
for i in range(1000):  
    MyModel.train_step(image, style_image)
plt.imshow(MyModel.tensor_to_image(image))
plt.show()

### Weight Exploration

In [ ]:
style_weights = [1e-1, 1e1, 1e2, 1e3, 1e4]
content_weights = [1e3, 1e4, 1e5, 1e6]

for style_weight in style_weights:
    
    for content_weight in content_weights:
        
        image = tf.Variable(content_image)
        
        for i in range(1000):
            
            MyModel.weight_exploration(image, style_weight, content_weight)
            
            if i % 999 == 0:
                print(f'Style weight: {style_weight}; content weight: {content_weight}. Image after {i} Trainingsteps:')
                plt.imshow(MyModel.tensor_to_image(image))
                plt.show()
    

## Perform Style-Transfer on a batch of images

In [ ]:
style_to_number = {
    'cubism': 0,
    'expressionism' : 1,
    'romanticism' : 2
}

def load_images(folder):

    path = '../input/art-movements/dataset/' + folder
    cubism_paths = glob.glob(path + '/cubism/*')
    expressionism_paths = glob.glob(path + '/expressionism/*')
    romanticism_paths = glob.glob(path + '/romanticism/*')
    
    combined_paths = [cubism_paths, expressionism_paths, romanticism_paths]

    art_styles = ['cubism', 'expressionnism', 'romanticism']
    images = []
    labels = []

    for i, art_style in enumerate(combined_paths):
        for image_path in art_style:
            image = load_img(image_path)     
            images.append(image)
            labels.append(i)

    return images, labels

def test_data_gen():
    for i, image in enumerate(test_images):
        yield image, test_labels[i]

In [ ]:
test_images, test_labels = load_images(folder = 'test')


test_ds = tf.data.Dataset.from_generator(test_data_gen, output_signature=(tf.TensorSpec(shape=(None, None, None, 3)),
                                                             tf.TensorSpec(shape=(), dtype=tf.int32))
                                                             )

In [ ]:
for images, labels in test_ds:
    print(labels)
    plt.imshow(MyModel.tensor_to_image(images))
    plt.show()
    break

In [ ]:
stylised_images = []
counter = 0

#perform the style transfer for every image in the test dataset
for art_image, label in test_ds:
    
    image = tf.Variable(load_img(content_path))
    
    for i in range(1000):
        
        MyModel.train_step(image, art_image)

        if (i == 999):

            print(f'-Image at step{i}')
            
            plt.subplot(1, 2, 1)
            imshow(art_image, 'Style Image')
            plt.subplot(2, 2, 2)
            plt.imshow(MyModel.tensor_to_image(image))
            plt.show()
            stylised_images.append(image)
        if (i==999):  
            plt.imshow(MyModel.tensor_to_image(image))
            plt.savefig(f'VGG_LR2_sw10_cw5e5_1500it_{label}_Image{counter}.png')
            plt.show()
    
    counter += 1

In [ ]:
for label in stylised_images:
    print (label)
    print (MyModel.tensor_to_image(label))
    break